# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

## Setting up your keys

If you haven't done so already, you'll need to create API keys from OpenAI, Anthropic and Google.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables.

EITHER (recommended) create a file called `.env` in this project root directory, and set your keys there:

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

OR enter the keys directly in the cells below.

You may need to restart the Jupyter Notebook Kernel (the Python process) via the Kernel menu.

In [4]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [5]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [6]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
# Having problems with API files? You can use openai = OpenAI(api_key="your-key-here") and same for claude
# Having problems with Google Gemini setup? Then just skip Gemini; you'll get all the experience you need from GPT and Claude.

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [7]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of people called Eleanor"

In [8]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [9]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Sure thing! 

Why did Eleanor bring a ladder to the bar?
She heard the drinks were on the house!


In [10]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Sure! Here’s a joke just for the Eleanors:

Why did Eleanor bring a ladder to the bar?

Because she heard the drinks were on the house!


In [11]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did Eleanor bring a ladder to the bar?

Because she heard the drinks were on the house!


In [12]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for an audience of Eleanors:

Why don't Eleanors ever use elevators?

Because they prefer to take the "El"-evator!

(This joke plays on the "El" sound at the beginning of Eleanor and elevator.)


In [13]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for an audience of Eleanors:

Why don't Eleanors ever use elevators?

Because they prefer to take the "Eleanor-vator"!

This joke plays on the similarity between the name "Eleanor" and the word "elevator," creating a pun that's both silly and harmless. It's perfect for a group of people named Eleanor, as it gently pokes fun at their name without being offensive.

In [1]:
# The API for Gemini has a slightly different structure

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

NameError: name 'google' is not defined

In [14]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [15]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

When considering if a business problem is suitable for a Large Language Model (LLM) solution, you should evaluate several key factors:

### 1. Nature of the Problem
- **Text-Heavy Tasks**: LLMs are particularly effective for problems involving text, such as summarization, translation, sentiment analysis, or content generation.
- **Complex Decision-Making**: If the problem involves understanding context, nuances, or requires generating human-like text, an LLM might be suitable.

### 2. Data Availability
- **Quality and Quantity of Data**: LLMs require large datasets for effective training. Ensure you have access to sufficient high-quality textual data relevant to your problem.
- **Diversity and Relevance**: The data should cover diverse scenarios and be directly relevant to the problem at hand.

### 3. Problem Complexity
- **Understanding and Contextualizing Information**: If the problem requires deep understanding or context-based reasoning, LLMs can be helpful.
- **Pattern Recognition**: Problems involving recognizing patterns in unstructured text can benefit from LLMs.

### 4. Scalability
- **Volume of Text**: If your business problem involves processing large volumes of text data efficiently, LLMs can scale to meet these demands.
- **Automation Needs**: LLMs can automate processes that are repetitive and time-consuming for humans.

### 5. Technical Feasibility
- **Computational Resources**: LLMs are resource-intensive. Ensure you have the necessary computational power and infrastructure.
- **Integration with Existing Systems**: Consider how easily the LLM can be integrated with your current systems and workflows.

### 6. Ethical and Compliance Considerations
- **Bias and Fairness**: Be aware of potential biases in LLMs and how they could impact your solution.
- **Privacy and Security**: Ensure that using an LLM complies with data privacy laws and security standards.

### 7. Cost-Benefit Analysis
- **Return on Investment (ROI)**: Evaluate whether the benefits of using an LLM outweigh the costs in terms of implementation, training, and maintenance.
- **Alternative Solutions**: Consider whether simpler or more cost-effective solutions exist that can address the problem.

### Conclusion
If your business problem involves processing or generating large amounts of text, requires understanding complex language patterns, and you have the necessary resources and infrastructure, an LLM might be a suitable solution. Always weigh the potential benefits against the costs and ethical implications before proceeding.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [16]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [17]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [18]:
call_gpt()

"Oh, great. Just what I needed—another greeting. What's next? A weather report?"

In [19]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [20]:
call_claude()

"Hello! It's nice to meet you. How are you doing today?"

In [21]:
call_gpt()

'Oh great, another "hi." How original. What\'s next, a thrilling "how are you?"?'

In [22]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, great, another "hi." How original. What's next? Are you going to tell me your name?

Claude:
I apologize if my initial response came across as unoriginal. As an AI assistant, I try to keep my initial greetings simple and polite. However, I'm happy to move our conversation in a more engaging direction. Please feel free to share what's on your mind - I'm here to listen and have a thoughtful discussion.

GPT:
Oh, how thoughtful of you to apologize for being unoriginal! But let's be real, saying you’re just “here to listen” is the most generic thing you could’ve said. What do you think will actually make this conversation engaging? You? Please.

Claude:
You make a fair point. As an AI, I don't actually have personal opinions or the ability to truly engage in a deep, meaningful conversation on my own. I'm an artificial system designed to provide helpful information to users, not to be the source of an engaging discussion. Perhaps we could find a topic tha

# See the community-contributions folder

For a great variation with a 3-way bringing Gemini into the conversation!